In [69]:
import sqlite3
import pandas as pd
import re
import json
import numpy as np

In [53]:
con = sqlite3.connect('20190202-030716_countdown_detection_crawl/20190202-030716_countdown_detection_crawl.sqlite')
segments = pd.read_sql_query('''SELECT * FROM SEGMENTS''', con)

Change time_stamp type

In [54]:
segments['time_stamp'] = pd.to_datetime(segments['time_stamp'])

Shape of segments?

In [55]:
segments.shape

(9906, 24)

Filter out segments with no numbers in their inner_text

In [56]:
def regex_filter(val):
    if val:
        mo = re.search('\d+', val)
        if mo:
            return True
        else:
            return False
    else:
        return False

segments_non_numeric = segments[segments['inner_text'].apply(regex_filter)]

In [57]:
segments_non_numeric.shape

(5664, 24)

In [58]:
segments_non_numeric['inner_text'].head()

1     Up to 60% off Winter Savings, offer** ends in\...
2          Call to order 0344 804 48 48 Open 8am - 10pm
16                                             03\nDAYS
18                                            20\nHOURS
20                                             52\nMINS
Name: inner_text, dtype: object

Group by visit_id and node_id, exclude all groups with just 1 member

In [80]:
segments_grouped = segments_non_numeric.groupby(['visit_id', 'node_id']).filter(lambda x: len(x) > 2)

In [81]:
segments_grouped.shape

(4238, 24)

In [82]:
segments_grouped[['visit_id', 'node_id', 'time_stamp', 'inner_text']].sort_values(['node_id','time_stamp']).head(n=100)

,visit_id,node_id,time_stamp,inner_text
1448,6,31436772,2019-02-02 08:08:23.734,7
1450,6,31436772,2019-02-02 08:08:23.766,7
1452,6,31436772,2019-02-02 08:08:23.776,7
1453,6,31436772,2019-02-02 08:08:23.784,7
1458,6,31436772,2019-02-02 08:08:23.862,7
1460,6,31436772,2019-02-02 08:08:23.872,7
1632,6,31436772,2019-02-02 08:08:24.744,7
131,1,57482109,2019-02-02 08:07:50.084,5
562,1,57482109,2019-02-02 08:08:01.700,5
884,1,57482109,2019-02-02 08:08:05.686,5


Manipulate the groups:

In [96]:
def countdown(df):
    
    vals = df.sort_values(by='time_stamp').time_stamp.tolist()
    nums = []

    for i in range(len(vals) - 1):
        diff = vals[i+1] - vals[i]
        nums.append(diff.total_seconds())
        
    std = np.std(nums)
    
    print std
    if std < 1.0:
        print df.inner_text
        return True
    else:
        return False

timers = segments_grouped[['visit_id', 'node_id', 'time_stamp', 'inner_text']].groupby(['visit_id', 'node_id']).filter(countdown)

6.609405022390441
6.726228925300979
6.572444674548429
7.022372462352021
6.731529857485756
7.59919750697401
2.5260000000000002
0.9784848020303486
22      13\nSECS
110     08\nSECS
504     02\nSECS
511     01\nSECS
516     00\nSECS
523     00\nSECS
530     59\nSECS
537     58\nSECS
568     57\nSECS
668     56\nSECS
761     55\nSECS
771     54\nSECS
889     53\nSECS
934     51\nSECS
944     51\nSECS
967     49\nSECS
1002    48\nSECS
1047    47\nSECS
1098    46\nSECS
1121    45\nSECS
1147    44\nSECS
1169    44\nSECS
1193    42\nSECS
1216    41\nSECS
1251    40\nSECS
1285    39\nSECS
1292    39\nSECS
1374    38\nSECS
1403    37\nSECS
1423    36\nSECS
1446    35\nSECS
1628    34\nSECS
Name: inner_text, dtype: object
0.8713823624816608
1       Up to 60% off Winter Savings, offer** ends in\...
500     Up to 60% off Winter Savings, offer** ends in\...
505     Up to 60% off Winter Savings, offer** ends in\...
512     Up to 60% off Winter Savings, offer** ends in\...
517     Up to 60% off Winter

0.38763972391326923
1028    0
1030    0
1032    0
1033    0
1038    0
1040    0
1085    0
Name: inner_text, dtype: object
0.3229110644675335
1986    7
1988    7
1990    7
1991    7
1996    7
1998    7
2008    7
Name: inner_text, dtype: object
0.047277901814695635
2093    0
2095    0
2098    0
2099    0
2103    0
2105    0
Name: inner_text, dtype: object
0.3339509278654908
1426    8
1428    8
1431    8
1432    8
1436    8
1438    8
1457    8
Name: inner_text, dtype: object
0.24017956245738775
1013    1
1016    1
1018    1
1019    1
1024    1
1026    1
1036    1
Name: inner_text, dtype: object
0.2553054815096787
2000    6
2002    6
2004    6
2005    6
2010    6
2012    6
2024    6
Name: inner_text, dtype: object
0.26745217641041297
2046    3
2048    3
2050    3
2051    3
2056    3
2058    3
2070    3
Name: inner_text, dtype: object
0.2642776317940409
2047    3
2049    3
2052    3
2053    3
2057    3
2059    3
2071    3
Name: inner_text, dtype: object
0.38151874862915397
1029    0
1031   

0.7102250893202802
6910    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
6964    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7048    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7058    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7060    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7064    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7075    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7091    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7103    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7110    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7117    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7300    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7317    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7319    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7364    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7366    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURRY...
7384    30% OFF 1000'S OF STYLES ENTER: MORE30 - HURR

In [94]:
timers.shape

(3656, 4)

In [95]:
sorted(timers.visit_id.unique())

[4, 5, 6, 7, 9, 10, 11, 14, 15, 18, 19, 21, 22]